In [7]:
import scipy as sp
import numpy as np
import pandas as pd
import random
import csv
import importlib
import Estimators as est

importlib.reload(est)

np.set_printoptions(precision=2)

d = 10
dist_type = "exp"
est_type = "sw"
N=1000000
total_repeat = 1
print("distribution=",dist_type, "estimitor=",est_type, 
      "total sample=",N, "total repeat num=",total_repeat)
x_max = 1
x_step_size = x_max/d
x_grid = x_step_size*np.arange(d)
print(x_grid)

eps_grid = 0.5*np.arange(1,6)
print(eps_grid)

if dist_type=='exp':
    x_q = np.exp(-3*x_grid)
    x_q = x_q/np.sum(x_q)
elif dist_type=='uni':
    x_q = np.ones(d)/d
else:
    print('warning: invaild distribution type!')
print(x_q)

filename = "data/%s_q_%d.csv" %(dist_type,d)
x_data = {'x': x_grid, 'q':x_q}
pd.DataFrame(x_data).to_csv(filename)
print(filename, 'saved')

idx_original = random.choices(np.arange(d),x_q,k=N)
sample_original = x_grid[idx_original]

for i in range(len(eps_grid)):
    eps= eps_grid[i]
    print('eps=%.2f'%(eps))

    if est_type == 'sw':
        a_grid, M = est.SquareWave(eps,x_grid)
    elif est_type == 'grr':
        a_grid, M = est.GenRandResp(eps, x_grid)
    else:
        print('warning: invalid estimator type!')

    filename = 'data/%s_M_%.2f_%d.csv'%(est_type,eps,d)
    pd.DataFrame(M).to_csv(filename)
    filename = 'data/%s_a_%.2f_%d.csv'%(est_type,eps,d)
    pd.DataFrame(a_grid).to_csv(filename)

    rand_num = np.zeros((N,d))
    for j in range(d):
        rand_num[:,j] = random.choices(a_grid,M[:,j],k=N)
    x_q_noisy = np.zeros(d)
    for k in range(total_repeat):
        idx_noise = random.choices(range(N),k=N)
        idx_pair = list(zip(idx_noise,idx_original))
        sample_perturbed = rand_num[tuple(zip(*idx_pair))]
        elements,counts = np.unique(sample_perturbed,return_counts=True)
        






distribution= exp estimitor= sw total sample= 1000000 total repeat num= 1
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
[0.5 1.  1.5 2.  2.5]
[0.27 0.2  0.15 0.11 0.08 0.06 0.05 0.03 0.02 0.02]
data/exp_q_10.csv saved
[0.27 0.2  0.15 0.11 0.08 0.06 0.04 0.03 0.02 0.02]
[0.27 0.2  0.15 0.11 0.08 0.06 0.05 0.03 0.02 0.02]
eps=0.50
[-0.3 -0.2 -0.1  0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1.
  1.1  1.2]
[0.06 0.06 0.07 0.07 0.07 0.08 0.08 0.07 0.06 0.06 0.06 0.05 0.05 0.05
 0.05 0.05]
1.0
eps=1.00
[-0.2 -0.1  0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1.   1.1]
[0.06 0.08 0.09 0.1  0.11 0.09 0.08 0.07 0.06 0.06 0.05 0.05 0.05 0.05]
0.9999999999999999
eps=1.50
[-0.1  0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1. ]
[0.09 0.12 0.14 0.12 0.1  0.08 0.07 0.07 0.06 0.06 0.05 0.05]
1.0
eps=2.00
[-0.1  0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1. ]
[0.09 0.13 0.16 0.13 0.1  0.08 0.07 0.06 0.05 0.05 0.04 0.04]
1.0
eps=2.50
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
[0.19 0